In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import math

/home/ubuntu/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_csv('test_data/test_data.csv')
#labels = pd.read_csv('test_data/test_labels.csv')

In [ ]:
dummies = pd.get_dummies(dataset[['D_63', 'D_64']])

In [ ]:
timeint = pd.to_datetime(dataset['S_2']).astype(int)

In [ ]:
timeint = (timeint-timeint.min())/(timeint.max()-timeint.min())

In [ ]:
dataset_ = dataset.drop(['D_63', 'D_64', 'S_2'], axis=1)

In [ ]:
dataset_ = pd.concat([dataset_, dummies, timeint], axis = 1)

In [ ]:
dataset_ = dataset_.fillna(0)

In [ ]:
dataset_ = dataset_.groupby(["customer_ID"], as_index=False).agg(list)

In [ ]:
def fillSeries(row):
    new_row = []
    size = len(row[1])
    if size < 13:
        for idx in range(len(row)):
            if idx == 0: new_row.append(row[idx])
            else: new_row.append([row[idx][0] for cnt in range(13 - size)] + row[idx])
    else:
        new_row = row
    return new_row

In [ ]:
dataset_ = dataset_.apply(fillSeries, axis=1)

In [ ]:
#dataset_ = dataset_.set_index('customer_ID').join(labels.set_index('customer_ID'))

In [ ]:
dataset_ = dataset_.reset_index(level=0)

In [ ]:
dataset_.to_pickle("./transformed_test_dataset")

: 